In [1]:
import pandas as pd
import numpy as np

Load Concrete date

In [2]:
concrete_data = pd.read_csv('https://cocl.us/concrete_data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


Check the size of the data

In [3]:
concrete_data.shape

(1030, 9)

Check columns and contents

In [4]:
concrete_data.describe

<bound method NDFrame.describe of       Cement  Blast Furnace Slag  Fly Ash  Water  Superplasticizer  \
0      540.0                 0.0      0.0  162.0               2.5   
1      540.0                 0.0      0.0  162.0               2.5   
2      332.5               142.5      0.0  228.0               0.0   
3      332.5               142.5      0.0  228.0               0.0   
4      198.6               132.4      0.0  192.0               0.0   
5      266.0               114.0      0.0  228.0               0.0   
6      380.0                95.0      0.0  228.0               0.0   
7      380.0                95.0      0.0  228.0               0.0   
8      266.0               114.0      0.0  228.0               0.0   
9      475.0                 0.0      0.0  228.0               0.0   
10     198.6               132.4      0.0  192.0               0.0   
11     198.6               132.4      0.0  192.0               0.0   
12     427.5                47.5      0.0  228.0        

Let's check if we are missing values

In [5]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Predictors are all columns exceft for Strength which is the target

In [6]:
concrete_data_cols = concrete_data.columns
predictors = concrete_data[concrete_data_cols[concrete_data_cols != 'Strength']]
target = concrete_data['Strength']

Normalize the data

In [7]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [8]:
n_cols = predictors_norm.shape[1] # number of predictors to be used to build the NN

Let's get started with Keras

In [9]:
import keras

Using TensorFlow backend.


In [10]:
from keras.models import Sequential
from keras.layers import Dense

Let's define a regression model, using 1 layer with 10 nodes, the ReLU activation function, adam as the optimizer and the mean squared error as the loss function

In [11]:
def regression_model():
    model = Sequential()
    model.add(Dense(10,activation='relu',input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Train and test the network

In [12]:
model = regression_model()
print(model.metrics_names)

Instructions for updating:
Colocations handled automatically by placer.
['loss']


Train the model using 50 epochs - loop 50 times reporting the mean squared errors

In [13]:
from sklearn.model_selection import train_test_split
scores = []
# fit the model
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.30, random_state=24)
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=1)
    ll = model.evaluate(X_test, y_test, verbose=0)
    scores.append(ll)
print(scores)
 

Instructions for updating:
Use tf.cast instead.
Train on 721 samples, validate on 309 samples
Epoch 1/50
721/721 [==============================] - 2s 3ms/step - loss: 1709.9924 - val_loss: 1240.5150
Epoch 2/50
721/721 [==============================] - 1s 1ms/step - loss: 1696.8351 - val_loss: 1231.0130
Epoch 3/50
721/721 [==============================] - 0s 691us/step - loss: 1683.6001 - val_loss: 1221.8995
Epoch 4/50
721/721 [==============================] - 0s 665us/step - loss: 1670.6145 - val_loss: 1213.1253
Epoch 5/50
721/721 [==============================] - 0s 571us/step - loss: 1657.1386 - val_loss: 1204.2531
Epoch 6/50
721/721 [==============================] - 0s 535us/step - loss: 1642.9023 - val_loss: 1195.5201
Epoch 7/50
721/721 [==============================] - 0s 540us/step - loss: 1628.0906 - val_loss: 1186.5686
Epoch 8/50
721/721 [==============================] - 0s 612us/step - loss: 1611.8916 - val_loss: 1177.4830
Epoch 9/50
721/721 [==========================

Report mean and standard deviation of fitting iterates

In [14]:
import statistics as st
print(scores)
print("Mean loss is %f" % st.mean(scores))
print("With standard deviation %f" % st.stdev(scores))

[271.5480720989141, 161.0375897552589, 117.1465923716721, 87.01620473522199, 69.65994586142136, 61.67059669371176, 57.90277131707151, 56.098864669552896, 55.152063215434744, 54.26930930622187, 54.51536165009039, 54.673847210831624, 54.35576834570629, 54.80741352710909, 54.63679276006507, 55.789498066825004, 55.694348467978074, 56.146591705025976, 56.67795261518855, 56.722393455628826, 55.828018176131266, 55.942752591228796, 56.16724213041534, 55.58635782038124, 55.53126238233449, 55.613636547693545, 55.617130254850416, 56.02176731689848, 56.417810631415605, 56.51692857711446, 56.71750568031879, 56.62053423637711, 56.97097954858082, 57.58942688630237, 57.891640227975195, 58.00410814501321, 58.39773350934766, 58.977428547386985, 59.52529236876849, 60.070640033117, 60.23718819726246, 61.415660376687654, 62.46852794511418, 63.24747999123385, 63.42688416663111, 64.16135928700271, 64.90224497526594, 65.70798698598126, 66.17277847833232, 66.14659376669084]
Mean loss is 66.468338
With standard

The mean for the normalized predictors is lower than Step A